# Descripción de los Archivos

## Archivo para Crear los CSV de Entrenamiento

Este archivo se usa para crear los CSV utilizados durante el entrenamiento. Se implementan funciones que extraen las características de las imágenes de entrenamiento y las guardan en diferentes CSV. Los CSV generados contienen:

- **Baseline RGB**: Valores originales RGB.
- **Características RGB downsampleado**: Valores RGB reducidos.
- **HOG (Histogram of Oriented Gradients)**: Características basadas en gradientes.
- **LBP (Local Binary Patterns)**: Características basadas en patrones binarios.
- **Todos juntos**: Combina todas las características anteriores.

Además, se crean versiones normalizadas y sin normalizar de cada uno.

---

In [2]:
# Ejemplo de uso
sat_folder = 'DataSet_1/roads/sat'
gt_folder = 'DataSet_1/roads/gt'
sat_folder_test = 'DataSet_1/test/sat/'
gt_folder_test = 'DataSet_1/test/gt/'


In [11]:
import os
from PIL import Image
import pandas as pd

def convert_images_to_csv(sat_folder, gt_folder, output_csv_path):
    """
    Lee las imágenes de sat_folder, guarda sus valores RGB en un CSV y usa las máscaras de gt_folder
    para agregar una etiqueta binaria (label) que indica si hay carretera o no.

    Args:
        sat_folder (str): Ruta a la carpeta que contiene las imágenes satelitales (RGB).
        gt_folder (str): Ruta a la carpeta que contiene las máscaras binarias (0 o 255).
        output_csv_path (str): Ruta del archivo CSV de salida.

    Returns:
        None
    """
    # Lista para almacenar los datos
    data = []

    # Obtenemos los nombres de las imágenes satelitales
    sat_images = [f for f in os.listdir(sat_folder) if f.endswith('.tiff')]

    for image_name in sat_images:
        # Construimos las rutas completas
        sat_image_path = os.path.join(sat_folder, image_name)
        gt_image_name = os.path.splitext(image_name)[0] + '.tif'
        gt_image_path = os.path.join(gt_folder, gt_image_name)

        # Verificamos si la máscara correspondiente existe
        if not os.path.exists(gt_image_path):
            print(f"Advertencia: No se encontró máscara para {image_name}, omitiendo...")
            continue

        try:
            # Cargamos la imagen satelital y la máscara
            sat_image = Image.open(sat_image_path).convert("RGB")
            gt_image = Image.open(gt_image_path).convert("L")

            # Redimensionamos la máscara para que coincida con la imagen satelital (si es necesario)
            if sat_image.size != gt_image.size:
                gt_image = gt_image.resize(sat_image.size, Image.NEAREST)

            # Convertimos las imágenes a listas de píxeles
            sat_pixels = list(sat_image.getdata())  # RGB
            gt_pixels = list(gt_image.getdata())   # Etiquetas (0 o 255)

            # Convertimos las etiquetas a binarias (0 o 1)
            gt_pixels = [1 if pixel > 0 else 0 for pixel in gt_pixels]

            # Almacenamos los datos (R, G, B, label)
            for rgb, label in zip(sat_pixels, gt_pixels):
                data.append([rgb[0], rgb[1], rgb[2], label])

        except Exception as e:
            print(f"Error al procesar {image_name}: {e}, omitiendo...")
            continue

    # Guardamos los datos en un archivo CSV
    df = pd.DataFrame(data, columns=['R', 'G', 'B', 'Label'])
    df.to_csv(output_csv_path, index=False)

    print(f"CSV generado con éxito en: {output_csv_path}")

In [12]:
csv = 'csv/RGBLabel.csv'

sat_folder = 'DataSet_1/roads/sat'
gt_folder = 'DataSet_1/roads/gt'

convert_images_to_csv(sat_folder,gt_folder,csv)

CSV generado con éxito en: csv/RGBLabel.csv


In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image

def convert_images_to_csv_with_downsampling(sat_folder, gt_folder, output_csv_path):
    # Lista para almacenar los datos de todas las imágenes
    dataset = []
    
    # Listar archivos disponibles en cada directorio para verificar problemas de nombres
    sat_files = sorted(os.listdir(sat_folder))
    gt_files = sorted(os.listdir(gt_folder))

    print("Imágenes satelitales encontradas:", sat_files)
    print("Imágenes de máscaras encontradas:", gt_files)

    for sat_file in sat_files:
        # Generar rutas para extensiones .tiff y .tif
        sat_path = os.path.join(sat_folder, sat_file)
        gt_path_tiff = os.path.join(gt_folder, sat_file)
        gt_path_tif = os.path.join(gt_folder, sat_file.replace('.tiff', '.tif'))
        
        # Verificar si existe alguna versión de la máscara
        if os.path.exists(gt_path_tiff):
            gt_path = gt_path_tiff
        elif os.path.exists(gt_path_tif):
            gt_path = gt_path_tif
        else:
            print(f"Advertencia: Falta la máscara para {sat_file}. Saltando este archivo.")
            continue
        
        # Cargar las imágenes
        try:
            satellite_image = Image.open(sat_path)
            mask_image = Image.open(gt_path)
        except Exception as e:
            print(f"Error al cargar {sat_file}: {e}")
            continue
        
        # Convertir las imágenes a arreglos numpy
        satellite_array = np.array(satellite_image)
        mask_array = np.array(mask_image)
        
        # Validar dimensiones
        if satellite_array.shape[:2] != mask_array.shape[:2]:
            print(f"Advertencia: Tamaño desigual para {sat_file}. Saltando este par de imágenes.")
            continue
        
        # Aplanar los arreglos
        pixels_rgb = satellite_array.reshape(-1, 3)  # Características RGB
        labels = mask_array.flatten()                # Etiquetas binarias
        
        # Combinar valores RGB y etiquetas
        image_data = np.column_stack((pixels_rgb, labels))
        dataset.append(image_data)
    
    if dataset:
        # Concatenar todos los datos en un solo conjunto
        dataset = np.vstack(dataset)
        
        # Convertir a DataFrame para manipulación más fácil
        df = pd.DataFrame(dataset, columns=['R', 'G', 'B', 'Label'])
        
        # Suponiendo que df es tu DataFrame original
        df['Label'] = df['Label'].replace({255: 1, 0: 0})
     
        roads = df[df['Label'] == 1]  
        non_roads = df[df['Label'] == 0] 
        
        
        # Realizar downsampling: seleccionar 3 píxeles clase 0 por cada clase 255
        non_roads_sampled = non_roads.sample(n=3 * len(roads), random_state=42, replace=False)
        
        # Combinar los píxeles seleccionados
        balanced_df = pd.concat([roads, non_roads_sampled])
        
        # Mezclar filas para garantizar aleatoriedad
        balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
        
        # Guardar el DataFrame balanceado en un CSV
        balanced_df.to_csv(output_csv_path, index=False)
        print(f"Dataset balanceado guardado en {output_csv_path}")
    else:
        print("Error: No se encontraron datos de imágenes válidos. Verifica los archivos en las carpetas 'sat' y 'gt'.")

In [4]:
csv = 'csv/downsampled.csv'

sat_folder = 'DataSet_1/roads/sat'
gt_folder = 'DataSet_1/roads/gt'

convert_images_to_csv_with_downsampling(sat_folder,gt_folder,csv)

Imágenes satelitales encontradas: ['10078675_15.tiff', '10228675_15.tiff', '10228705_15.tiff', '10228720_15.tiff', '10228735_15.tiff', '10228750_15.tiff', '10378675_15.tiff', '10378690_15.tiff', '10378705_15.tiff', '10378720_15.tiff', '10378750_15.tiff', '10378765_15.tiff', '10528675_15.tiff', '10528690_15.tiff', '10528735_15.tiff', '10528750_15.tiff']
Imágenes de máscaras encontradas: ['10078675_15.tif', '10228675_15.tif', '10228705_15.tif', '10228720_15.tif', '10228735_15.tif', '10228750_15.tif', '10378675_15.tif', '10378690_15.tif', '10378705_15.tif', '10378720_15.tif', '10378750_15.tif', '10378765_15.tif', '10528675_15.tif', '10528690_15.tif', '10528735_15.tif', '10528750_15.tif']
Dataset balanceado guardado en csv/downsampled.csv


In [75]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2gray

def csvhog(sat_folder, gt_folder, output_csv_path):
    # Lista para almacenar los datos de todas las imágenes
    dataset = []
    
    # Listar archivos disponibles en cada directorio para verificar problemas de nombres
    sat_files = sorted(os.listdir(sat_folder))
    gt_files = sorted(os.listdir(gt_folder))

    print("Imágenes satelitales encontradas:", sat_files)
    print("Imágenes de máscaras encontradas:", gt_files)

    for sat_file in sat_files:
        # Generar rutas para extensiones .tiff y .tif
        sat_path = os.path.join(sat_folder, sat_file)
        gt_path_tiff = os.path.join(gt_folder, sat_file)
        gt_path_tif = os.path.join(gt_folder, sat_file.replace('.tiff', '.tif'))
        
        # Verificar si existe alguna versión de la máscara
        if os.path.exists(gt_path_tiff):
            gt_path = gt_path_tiff
        elif os.path.exists(gt_path_tif):
            gt_path = gt_path_tif
        else:
            print(f"Advertencia: Falta la máscara para {sat_file}. Saltando este archivo.")
            continue
        
        # Cargar las imágenes
        try:
            satellite_image = Image.open(sat_path)
            mask_image = Image.open(gt_path)
        except Exception as e:
            print(f"Error al cargar {sat_file}: {e}")
            continue
        
        # Convertir las imágenes a arreglos numpy
        satellite_array = np.array(satellite_image)
        mask_array = np.array(mask_image)
        
        # Validar dimensiones
        if satellite_array.shape[:2] != mask_array.shape[:2]:
            print(f"Advertencia: Tamaño desigual para {sat_file}. Saltando este par de imágenes.")
            continue
        
        # Convertir la imagen satelital a escala de grises para HOG
        gray_image = rgb2gray(satellite_array)

        # Calcular descriptores HOG
        fd, hog_image = hog(
            gray_image,
            pixels_per_cell=(8, 8),
            cells_per_block=(1, 1),
            orientations=9,
            visualize=True,
            feature_vector=False
        )


        # Aplanar los arreglos
        pixels_hog = hog_image.flatten()     # Descriptores HOG
        labels = mask_array.flatten()               # Etiquetas binarias
        
        # Combinar valores HOG y etiquetas
        image_data = np.column_stack((pixels_hog, labels))
        dataset.append(image_data)
    
    if dataset:
        # Concatenar todos los datos en un solo conjunto
        dataset = np.vstack(dataset)
        
        # Convertir a DataFrame para manipulación más fácil
        columns = ['HOG'] + ['Label']
        df = pd.DataFrame(dataset, columns=columns)
        
        # Suponiendo que df es tu DataFrame original
        df['Label'] = df['Label'].replace({255: 1, 0: 0})
        
        roads = df[df['Label'] == 1]  
        non_roads = df[df['Label'] == 0] 
        
        # Realizar downsampling: seleccionar 3 píxeles clase 0 por cada clase 255
        non_roads_sampled = non_roads.sample(n=3 * len(roads), random_state=42, replace=False)
        
        # Combinar los píxeles seleccionados
        balanced_df = pd.concat([roads, non_roads_sampled])
        
        # Mezclar filas para garantizar aleatoriedad
        balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
        
        # Guardar el DataFrame balanceado en un CSV
        balanced_df.to_csv(output_csv_path, index=False)
        print(f"Dataset balanceado guardado en {output_csv_path}")
    else:
        print("Error: No se encontraron datos de imágenes válidos. Verifica los archivos en las carpetas 'sat' y 'gt'.")


In [77]:
csv = 'csv/hog.csv'

sat_folder = 'DataSet_1/roads/sat'
gt_folder = 'DataSet_1/roads/gt'

csvhog(sat_folder,gt_folder,csv)

Imágenes satelitales encontradas: ['10078675_15.tiff', '10228675_15.tiff', '10228705_15.tiff', '10228720_15.tiff', '10228735_15.tiff', '10228750_15.tiff', '10378675_15.tiff', '10378690_15.tiff', '10378705_15.tiff', '10378720_15.tiff', '10378750_15.tiff', '10378765_15.tiff', '10528675_15.tiff', '10528690_15.tiff', '10528735_15.tiff', '10528750_15.tiff']
Imágenes de máscaras encontradas: ['10078675_15.tif', '10228675_15.tif', '10228705_15.tif', '10228720_15.tif', '10228735_15.tif', '10228750_15.tif', '10378675_15.tif', '10378690_15.tif', '10378705_15.tif', '10378720_15.tif', '10378750_15.tif', '10378765_15.tif', '10528675_15.tif', '10528690_15.tif', '10528735_15.tif', '10528750_15.tif']
Dataset balanceado guardado en csv/hog.csv


In [78]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2gray

def csvlbp(sat_folder, gt_folder, output_csv_path):
    # Lista para almacenar los datos de todas las imágenes
    dataset = []

     # Parámetros para LBP
    P = 8  # Número de puntos
    R = 1  # Radio
    lbp_max_possible = P + 2  # P+1 (uniformes) + 1 (no uniformes)
    lbp_min_possible = 0      # El valor mínimo siempre es 0

    # Listar archivos disponibles en cada directorio para verificar problemas de nombres
    sat_files = sorted(os.listdir(sat_folder))
    gt_files = sorted(os.listdir(gt_folder))

    print("Imágenes satelitales encontradas:", sat_files)
    print("Imágenes de máscaras encontradas:", gt_files)

    for sat_file in sat_files:
        # Generar rutas para extensiones .tiff y .tif
        sat_path = os.path.join(sat_folder, sat_file)
        gt_path_tiff = os.path.join(gt_folder, sat_file)
        gt_path_tif = os.path.join(gt_folder, sat_file.replace('.tiff', '.tif'))
        
        # Verificar si existe alguna versión de la máscara
        if os.path.exists(gt_path_tiff):
            gt_path = gt_path_tiff
        elif os.path.exists(gt_path_tif):
            gt_path = gt_path_tif
        else:
            print(f"Advertencia: Falta la máscara para {sat_file}. Saltando este archivo.")
            continue
        
        # Cargar las imágenes
        try:
            satellite_image = Image.open(sat_path)
            mask_image = Image.open(gt_path)
        except Exception as e:
            print(f"Error al cargar {sat_file}: {e}")
            continue
        
        # Convertir las imágenes a arreglos numpy
        satellite_array = np.array(satellite_image)
        mask_array = np.array(mask_image)
        
        # Validar dimensiones
        if satellite_array.shape[:2] != mask_array.shape[:2]:
            print(f"Advertencia: Tamaño desigual para {sat_file}. Saltando este par de imágenes.")
            continue
        
        # Convertir la imagen satelital a escala de grises para HOG
        gray_image = rgb2gray(satellite_array)

        lbp = local_binary_pattern(gray_image, P=P, R=R, method="uniform")

        # Aplanar los arreglos
        pixels_lbp = lbp.flatten()

        labels = mask_array.flatten()               # Etiquetas binarias
        
        # Combinar valores HOG y etiquetas
        image_data = np.column_stack((pixels_lbp, labels))
        dataset.append(image_data)
    
    if dataset:
        # Concatenar todos los datos en un solo conjunto
        dataset = np.vstack(dataset)
        
        # Convertir a DataFrame para manipulación más fácil
        columns = ['LBP'] + ['Label']
        df = pd.DataFrame(dataset, columns=columns)
        
        df['Label'] = df['Label'].replace({255: 1, 0: 0})
        
        roads = df[df['Label'] == 1] 
        non_roads = df[df['Label'] == 0] 
        
        
        # Realizar downsampling: seleccionar 3 píxeles clase 0 por cada clase 255
        non_roads_sampled = non_roads.sample(n=3 * len(roads), random_state=42, replace=False)
        
        # Combinar los píxeles seleccionados
        balanced_df = pd.concat([roads, non_roads_sampled])
        
        # Mezclar filas para garantizar aleatoriedad
        balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
        
        # Guardar el DataFrame balanceado en un CSV
        balanced_df.to_csv(output_csv_path, index=False)
        print(f"Dataset balanceado guardado en {output_csv_path}")
    else:
        print("Error: No se encontraron datos de imágenes válidos. Verifica los archivos en las carpetas 'sat' y 'gt'.")


In [79]:
csv = 'csv/lbp.csv'

sat_folder = 'DataSet_1/roads/sat'
gt_folder = 'DataSet_1/roads/gt'

csvlbp(sat_folder,gt_folder,csv)

Imágenes satelitales encontradas: ['10078675_15.tiff', '10228675_15.tiff', '10228705_15.tiff', '10228720_15.tiff', '10228735_15.tiff', '10228750_15.tiff', '10378675_15.tiff', '10378690_15.tiff', '10378705_15.tiff', '10378720_15.tiff', '10378750_15.tiff', '10378765_15.tiff', '10528675_15.tiff', '10528690_15.tiff', '10528735_15.tiff', '10528750_15.tiff']
Imágenes de máscaras encontradas: ['10078675_15.tif', '10228675_15.tif', '10228705_15.tif', '10228720_15.tif', '10228735_15.tif', '10228750_15.tif', '10378675_15.tif', '10378690_15.tif', '10378705_15.tif', '10378720_15.tif', '10378750_15.tif', '10378765_15.tif', '10528675_15.tif', '10528690_15.tif', '10528735_15.tif', '10528750_15.tif']
Dataset balanceado guardado en csv/lbp.csv


In [98]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2gray

def csvall(sat_folder, gt_folder, output_csv_path):
    # Lista para almacenar los datos de todas las imágenes
    dataset = []

     # Parámetros para LBP
    P = 8  # Número de puntos
    R = 1  # Radio
    lbp_max_possible = P + 2  # P+1 (uniformes) + 1 (no uniformes)
    lbp_min_possible = 0      # El valor mínimo siempre es 0

    # Listar archivos disponibles en cada directorio para verificar problemas de nombres
    sat_files = sorted(os.listdir(sat_folder))
    gt_files = sorted(os.listdir(gt_folder))

    print("Imágenes satelitales encontradas:", sat_files)
    print("Imágenes de máscaras encontradas:", gt_files)

    for sat_file in sat_files:
        # Generar rutas para extensiones .tiff y .tif
        sat_path = os.path.join(sat_folder, sat_file)
        gt_path_tiff = os.path.join(gt_folder, sat_file)
        gt_path_tif = os.path.join(gt_folder, sat_file.replace('.tiff', '.tif'))
        
        # Verificar si existe alguna versión de la máscara
        if os.path.exists(gt_path_tiff):
            gt_path = gt_path_tiff
        elif os.path.exists(gt_path_tif):
            gt_path = gt_path_tif
        else:
            print(f"Advertencia: Falta la máscara para {sat_file}. Saltando este archivo.")
            continue
        
        # Cargar las imágenes
        try:
            satellite_image = Image.open(sat_path)
            mask_image = Image.open(gt_path)
        except Exception as e:
            print(f"Error al cargar {sat_file}: {e}")
            continue
        
        # Convertir las imágenes a arreglos numpy
        satellite_array = np.array(satellite_image)
        mask_array = np.array(mask_image)
        
        # Validar dimensiones
        if satellite_array.shape[:2] != mask_array.shape[:2]:
            print(f"Advertencia: Tamaño desigual para {sat_file}. Saltando este par de imágenes.")
            continue
        
        # Convertir la imagen satelital a escala de grises para HOG
        gray_image = rgb2gray(satellite_array)

        lbp = local_binary_pattern(gray_image, P=P, R=R, method="uniform")


        # Calcular descriptores HOG
        fd, hog_image = hog(
            gray_image,
            pixels_per_cell=(8, 8),
            cells_per_block=(1, 1),
            orientations=9,
            visualize=True,
            feature_vector=False
        )

         # Aplanar los arreglos
        pixels_rgb = satellite_array.reshape(-1, 3)  # Características RGB
        pixels_lbp = lbp.flatten()
        pixels_hog = hog_image.flatten()     # Descriptores HOG
        labels = mask_array.flatten()               # Etiquetas binarias
        
        # Combinar valores HOG y etiquetas
        image_data = np.column_stack((pixels_rgb,pixels_lbp, pixels_hog,labels))
        dataset.append(image_data)
    
    if dataset:
        # Concatenar todos los datos en un solo conjunto
        dataset = np.vstack(dataset)
        
        # Convertir a DataFrame para manipulación más fácil
        columns = ['R','G','B'] + ['HOG']+['LBP'] + ['Label']
        df = pd.DataFrame(dataset, columns=columns)
        
        # Suponiendo que df es tu DataFrame original
        df['Label'] = df['Label'].replace({255: 1, 0: 0})
        
        # Ahora puedes seguir utilizando 'roads' y 'non_roads'
        roads = df[df['Label'] == 1]  # Mantener todos los píxeles clase 1
        non_roads = df[df['Label'] == 0]  # Mantener todos los píxeles clase 0
        
        
        # Realizar downsampling: seleccionar 3 píxeles clase 0 por cada clase 255
        non_roads_sampled = non_roads.sample(n=3 * len(roads), random_state=42, replace=False)
        
        # Combinar los píxeles seleccionados
        balanced_df = pd.concat([roads, non_roads_sampled])
        
        # Mezclar filas para garantizar aleatoriedad
        balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
        
        # Guardar el DataFrame balanceado en un CSV
        balanced_df.to_csv(output_csv_path, index=False)
        print(f"Dataset balanceado guardado en {output_csv_path}")
    else:
        print("Error: No se encontraron datos de imágenes válidos. Verifica los archivos en las carpetas 'sat' y 'gt'.")


## Este extrae todas las características y también normaliza

In [126]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog, local_binary_pattern
from skimage.color import rgb2gray

def csvall(sat_folder, gt_folder, output_csv_path):
    # Lista para almacenar los datos de todas las imágenes
    dataset = []

    # Parámetros para LBP
    P = 8  # Número de puntos
    R = 1  # Radio

    # Listar archivos disponibles en cada directorio para verificar problemas de nombres
    sat_files = sorted(os.listdir(sat_folder))
    gt_files = sorted(os.listdir(gt_folder))

    print("Imágenes satelitales encontradas:", sat_files)
    print("Imágenes de máscaras encontradas:", gt_files)

    for sat_file in sat_files:
        # Generar rutas para extensiones .tiff y .tif
        sat_path = os.path.join(sat_folder, sat_file)
        gt_path_tiff = os.path.join(gt_folder, sat_file)
        gt_path_tif = os.path.join(gt_folder, sat_file.replace('.tiff', '.tif'))

        # Verificar si existe alguna versión de la máscara
        if os.path.exists(gt_path_tiff):
            gt_path = gt_path_tiff
        elif os.path.exists(gt_path_tif):
            gt_path = gt_path_tif
        else:
            print(f"Advertencia: Falta la máscara para {sat_file}. Saltando este archivo.")
            continue

        # Cargar las imágenes
        try:
            satellite_image = Image.open(sat_path)
            mask_image = Image.open(gt_path)
        except Exception as e:
            print(f"Error al cargar {sat_file}: {e}")
            continue

        # Convertir las imágenes a arreglos numpy
        satellite_array = np.array(satellite_image)
        mask_array = np.array(mask_image)

        # Validar dimensiones
        if satellite_array.shape[:2] != mask_array.shape[:2]:
            print(f"Advertencia: Tamaño desigual para {sat_file}. Saltando este par de imágenes.")
            continue

        # Convertir la imagen satelital a escala de grises para HOG
        gray_image = rgb2gray(satellite_array)

        lbp = local_binary_pattern(gray_image, P=P, R=R, method="uniform")

        # Calcular descriptores HOG
        fd, hog_image = hog(
            gray_image,
            pixels_per_cell=(8, 8),
            cells_per_block=(1, 1),
            orientations=9,
            visualize=True,
            feature_vector=False
        )

        # Aplanar los arreglos
        pixels_rgb = satellite_array.reshape(-1, 3) / 255.0  # Características RGB escaladas a [0, 1]
        pixels_lbp = lbp.flatten()
        pixels_hog = hog_image.flatten()  # Descriptores HOG
        labels = mask_array.flatten()  # Etiquetas binarias

        # Combinar valores HOG y etiquetas
        image_data = np.column_stack((pixels_rgb, pixels_lbp, pixels_hog, labels))
        dataset.append(image_data)

    if dataset:
        # Concatenar todos los datos en un solo conjunto
        dataset = np.vstack(dataset)

        # Normalizar características entre 0 y 1
        features = dataset[:, :-1]  # Todas las columnas excepto la última (etiquetas)
        labels = dataset[:, -1]     # Última columna

        min_vals = np.min(features, axis=0)
        max_vals = np.max(features, axis=0)

        normalized_features = (features - min_vals) / (max_vals - min_vals)

        # Reconstruir el dataset normalizado
        normalized_dataset = np.column_stack((normalized_features, labels))

        # Convertir a DataFrame para manipulación más fácil
        columns = ['R', 'G', 'B'] + ['HOG'] + ['LBP', 'Label']
        df = pd.DataFrame(normalized_dataset, columns=columns)

        # Ajustar etiquetas (255 a 1, 0 a 0)
        df['Label'] = df['Label'].replace({255: 1, 0: 0})

        # Realizar downsampling
        roads = df[df['Label'] == 1]  # Clase 1
        non_roads = df[df['Label'] == 0]  # Clase 0

        # Seleccionar 3 píxeles clase 0 por cada clase 1
        non_roads_sampled = non_roads.sample(n=3 * len(roads), random_state=42, replace=False)

        # Combinar los píxeles seleccionados
        balanced_df = pd.concat([roads, non_roads_sampled])

        # Mezclar filas para garantizar aleatoriedad
        balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

        # Guardar el DataFrame balanceado en un CSV
        balanced_df.to_csv(output_csv_path, index=False)
        print(f"Dataset balanceado guardado en {output_csv_path}")
    else:
        print("Error: No se encontraron datos de imágenes válidos. Verifica los archivos en las carpetas 'sat' y 'gt'.")


In [127]:
csv = 'csv/all.csv'

sat_folder = 'DataSet_1/roads/sat'
gt_folder = 'DataSet_1/roads/gt'

csvall(sat_folder,gt_folder,csv)

Imágenes satelitales encontradas: ['10078675_15.tiff', '10228675_15.tiff', '10228705_15.tiff', '10228720_15.tiff', '10228735_15.tiff', '10228750_15.tiff', '10378675_15.tiff', '10378690_15.tiff', '10378705_15.tiff', '10378720_15.tiff', '10378750_15.tiff', '10378765_15.tiff', '10528675_15.tiff', '10528690_15.tiff', '10528735_15.tiff', '10528750_15.tiff']
Imágenes de máscaras encontradas: ['10078675_15.tif', '10228675_15.tif', '10228705_15.tif', '10228720_15.tif', '10228735_15.tif', '10228750_15.tif', '10378675_15.tif', '10378690_15.tif', '10378705_15.tif', '10378720_15.tif', '10378750_15.tif', '10378765_15.tif', '10528675_15.tif', '10528690_15.tif', '10528735_15.tif', '10528750_15.tif']
Dataset balanceado guardado en csv/all.csv
